In [1]:
import osmnx as ox
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
import streamlit as st
from streamlit_folium import folium_static
import matplotlib.pyplot as plt 
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
from prophet import Prophet
from fbprophet.plot import plot_plotly
import plotly.offline as py

In [3]:
df_acum= pd.read_excel('df_definitivo.xlsx')

In [4]:
df_acum

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,Precio gasóleo A,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción
0,ALBACETE,ABENGIBRE,ABENGIBRE,2250,"AVENIDA CASTILLA LA MANCHA, 26","-1,539167","39,211417","1,799",No disponible,No disponible,"1,829",No disponible,Nº 10.935,L-D: 07:00-22:00,04-04-2022
1,ALBACETE,ALATOZ,ALATOZ,2152,"CR CM-332, 46,4","-1,346083","39,100389","1,869",No disponible,"2,000","1,950","2,000",REPSOL,L-D: 7:00-23:00,04-04-2022
2,ALBACETE,ALBACETE,ALBACETE,2001,"CALLE FEDERICO GARCIA LORCA, 5","-1,846361","38,997722",No disponible,No disponible,No disponible,"1,889",No disponible,CEPSA,L-S: 05:00-23:00,04-04-2022
3,ALBACETE,ALBACETE,ALBACETE,2001,"AVENIDA 1º DE MAYO, S/N","-1,868500","38,985667","1,809",No disponible,"1,899","1,829","1,859",CARREFOUR,L-S: 08:00-22:00; D: 09:00-21:00,04-04-2022
4,ALBACETE,ALBACETE,ALBACETE,2001,CALLE PRINCIPE DE ASTURIAS (POLÍGONO DE ROMICA...,"-1,832000","39,054694","1,769",No disponible,"1,911","1,849","1,939",BP ROMICA,L-D: 06:00-21:30,04-04-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233145,ZARAGOZA,Zaragoza,ZARAGOZA,50197,"RONDA FERIA DE MUESTRAS, DE LA, 20","-0,986389","41,637944",No disponible,No disponible,No disponible,"1,889",No disponible,IDS,L-D: 24H,27-04-2022
233146,ZARAGOZA,Zaragoza,ZARAGOZA,50197,"POLIGONO CALLE TURIASO, 31","-0,974194","41,645722",No disponible,No disponible,No disponible,"1,889",No disponible,NOIL,L-D: 24H,27-04-2022
233147,ZARAGOZA,Zuera,ZUERA,50800,"CARRETERA N-330A KM. 24,000","-0,794000","41,857000","1,819",No disponible,"1,949","1,849","1,919",REPSOL,L-D: 06:00-22:00,27-04-2022
233148,ZARAGOZA,Zuera,ZUERA,50800,"CALLE AUSTRIA, 11,17","-0,782833","41,883583","1,799",No disponible,No disponible,"1,809","1,849",COOPERATIVA SAN LICER,L-D: 24H,27-04-2022


In [5]:
price_today= pd.read_excel('price_today.xlsx')


In [6]:
price_today = price_today.sort_values('Distance')
price_head=price_today.head(10)
price_head

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,Precio gasóleo A,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción,Latitud_partida,Longitud_partida,mercator_start,mercator_finish,Distance
6717,MADRID,Madrid,MADRID,28029,"CL MONFORTE DE LEMOS, 7",-3.691194,40.477556,"1,879","1,959","2,011","1,909","1,989",CEPSA,L-D: 07:00-23:00,27-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4505940.922476215 -411186.3653054542),1123.707543
6647,MADRID,Madrid,MADRID,28046,"CL PASEO CASTELLANA, 276",-3.686333,40.474833,"1,879",No disponible,"1,999","1,909","1,999",REPSOL,L-D: 24H,27-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4505637.799502784 -410644.1178619877),1165.060834
6716,MADRID,Madrid,MADRID,28029,"AVENIDA MONFORTE DE LEMOS, 10",-3.697889,40.479861,"1,879",No disponible,No disponible,"1,899","1,949",REPSOL,L-V: 07:00-22:00; S: 09:00-15:00,27-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4506197.513902494 -411933.2014001492),1599.801758
6691,MADRID,Madrid,MADRID,28036,"AVENIDA PÍO XII, 98",-3.675528,40.472556,"1,879",No disponible,"1,999","1,909","1,999",REPSOL,L-D: 24H,27-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4505384.325022249 -409438.8242909524),1725.549903
6730,MADRID,Madrid,MADRID,28034,"CALLE CARDENAL HERRERA ORIA, 81",-3.700583,40.489056,"1,879",No disponible,"2,019","1,899","1,979",SHELL,L-D: 24H,27-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4507221.096620337 -412233.7222499723),1840.447896
6728,MADRID,Madrid,MADRID,28034,"CL NUESTRA SEÑORA DE VALVERDE, 175",-3.685333,40.503444,"1,859",No disponible,"1,989","1,899","1,969",REPSOL,L-D: 06:00-22:00,27-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4508822.761453873 -410532.5676344641),2038.817755
6690,MADRID,Madrid,MADRID,28036,"CL MATEO INURRIA, 4",-3.685750,40.466667,"1,879","1,929","2,029","1,909","1,999",REPSOL,L-D: 24H,27-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4504728.764540968 -410579.0840640963),2061.372927
6726,MADRID,Madrid,MADRID,28034,"CARRETERA COLMENAR VIEJO KM. 10,5",-3.697806,40.499500,"1,870",No disponible,"2,010","1,900","1,980",REPSOL,L-D: 24H,27-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4508383.717382183 -411923.9426059921),2176.957616
6703,MADRID,Madrid,MADRID,28050,"AVENIDA MANOTERAS, 34",-3.665083,40.487861,"1,879",No disponible,"2,059","1,899","1,999",CEPSA,L-D: 06:00-23:59,27-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4507088.06982884 -408273.702460632),2192.722679
6727,MADRID,Madrid,MADRID,28034,"CARRETERA COLMENAR VIEJO KM. 10,5",-3.697972,40.500000,"1,870",No disponible,No disponible,"1,900","1,980",REPSOL,L-D: 24H,27-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4508439.377127579 -411942.460195173),2230.519888


In [7]:
#for j in range(10) 
#meter aqui todo el código con el número 5 pero con i

In [8]:
df_predicción=[]
direccion_gasolineras=list(price_head['Dirección'].unique())
for i in direccion_gasolineras:
    df_predicción.append(df_acum[df_acum.Dirección.isin([i])])
df_predicción[0]

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,Precio gasóleo A,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción
6693,MADRID,MADRID,MADRID,28029,"CL MONFORTE DE LEMOS, 7","-3,691194","40,477556","1,864","1,924","1,996","1,869","1,969",CEPSA,L-D: 07:00-23:00,04-04-2022
18316,MADRID,MADRID,MADRID,28029,"CL MONFORTE DE LEMOS, 7","-3,691194","40,477556","1,864","1,924","1,996","1,885","1,985",CEPSA,L-D: 07:00-23:00,05-04-2022
29963,MADRID,MADRID,MADRID,28029,"CL MONFORTE DE LEMOS, 7","-3,691194","40,477556","1,879","1,939","2,011","1,849","1,949",CEPSA,L-D: 07:00-23:00,09-04-2022
41610,MADRID,MADRID,MADRID,28029,"CL MONFORTE DE LEMOS, 7","-3,691194","40,477556","1,869","1,929","2,001","1,849","1,949",CEPSA,L-D: 07:00-23:00,10-04-2022
53275,MADRID,MADRID,MADRID,28029,"CL MONFORTE DE LEMOS, 7","-3,691194","40,477556","1,869","1,929","2,001","1,849","1,949",CEPSA,L-D: 07:00-23:00,11-04-2022
64959,MADRID,MADRID,MADRID,28029,"CL MONFORTE DE LEMOS, 7","-3,691194","40,477556","1,879","1,959","2,011","1,855","1,935",CEPSA,L-D: 07:00-23:00,12-04-2022
76659,MADRID,MADRID,MADRID,28029,"CL MONFORTE DE LEMOS, 7","-3,691194","40,477556","1,894","1,974","2,026","1,879","1,959",CEPSA,L-D: 07:00-23:00,14-04-2022
88349,MADRID,MADRID,MADRID,28029,"CL MONFORTE DE LEMOS, 7","-3,691194","40,477556","1,884","1,964","2,016","1,875","1,955",CEPSA,L-D: 07:00-23:00,15-04-2022
100020,MADRID,Madrid,MADRID,28029,"CL MONFORTE DE LEMOS, 7","-3,691194","40,477556","1,894","1,974","2,026","1,895","1,975",CEPSA,L-D: 07:00-23:00,16-04-2022
111677,MADRID,Madrid,MADRID,28029,"CL MONFORTE DE LEMOS, 7","-3,691194","40,477556","1,904","1,984","2,036","1,895","1,975",CEPSA,L-D: 07:00-23:00,17-04-2022


In [9]:
df_predicción[5]['Precio gasolina 95 E5'] = df_predicción[5]['Precio gasolina 95 E5'].apply(lambda x: x.replace(',','.'))
df_predicción[5]['Precio gasóleo A'] = df_predicción[5]['Precio gasóleo A'].apply(lambda x: x.replace(',','.'))
df_predicción[5]['Precio gasolina 95 E5']=df_predicción[5]['Precio gasolina 95 E5'].astype('float64')
df_predicción[5]['Precio gasóleo A']=df_predicción[5]['Precio gasóleo A'].astype('float64')

In [10]:
df_prophet_gasolina = pd.DataFrame()
df_prophet_gasolina['ds'] = df_predicción[5]['Fecha de extracción']
df_prophet_gasolina['y'] = df_predicción[5]['Precio gasolina 95 E5']
df_prophet_gasolina.head()

,ds,y
6706,04-04-2022,1.809
18330,05-04-2022,1.809
29975,09-04-2022,1.809
41621,10-04-2022,1.809
53289,11-04-2022,1.809


In [11]:
format_data = '%d-%m-%Y'
df_prophet_gasolina['ds']=df_prophet_gasolina['ds'].apply(lambda x: datetime.datetime.strptime(x,format_data))



In [12]:
df_prophet_gasolina.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 6706 to 228235
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      20 non-null     datetime64[ns]
 1   y       20 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 480.0 bytes


In [13]:

#df_prophet_gasolina['ds'] = pd.to_datetime(df_grafico[0]['Fecha de extracción']) 

#df_prophet_gasolina['ds'] = datetime.datetime.strptime(df_prophet_gasolina['ds'],'%Y/%m/%d')


df_prophet_gasolina.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 6706 to 228235
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      20 non-null     datetime64[ns]
 1   y       20 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 480.0 bytes


In [14]:
df_prophet_gasolina

,ds,y
6706,2022-04-04,1.809
18330,2022-04-05,1.809
29975,2022-04-09,1.809
41621,2022-04-10,1.809
53289,2022-04-11,1.809
64970,2022-04-12,1.809
76668,2022-04-14,1.829
88362,2022-04-15,1.849
100032,2022-04-16,1.859
111689,2022-04-17,1.859


In [15]:
#ax = df_prophet_gasolina.set_index('ds').plot(figsize=(12, 8))
#ax.set_ylabel('Price')
#ax.set_xlabel('Date')

#plt.show()

In [16]:
my_model = Prophet(interval_width=0.8)


In [17]:
my_model.fit(df_prophet_gasolina)


2022-04-27 20:28:14.874 INFO    prophet: Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
2022-04-27 20:28:14.876 INFO    prophet: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2022-04-27 20:28:14.905 INFO    prophet: n_changepoints greater than number of observations. Using 15.



Initial log joint probability = -10.4715
Iteration  1. Log joint probability =    46.8663. Improved by 57.3378.
Iteration  2. Log joint probability =    63.2105. Improved by 16.3442.
Iteration  3. Log joint probability =    74.9051. Improved by 11.6946.
Iteration  4. Log joint probability =    83.9092. Improved by 9.00409.
Iteration  5. Log joint probability =    90.0665. Improved by 6.1573.
Iteration  6. Log joint probability =    100.719. Improved by 10.6527.
Iteration  7. Log joint probability =    101.828. Improved by 1.10926.
Iteration  8. Log joint probability =    102.503. Improved by 0.674677.
Iteration  9. Log joint probability =    103.395. Improved by 0.891868.
Iteration 10. Log joint probability =    104.607. Improved by 1.21159.
Iteration 11. Log joint probability =    107.474. Improved by 2.86716.
Iteration 12. Log joint probability =    107.729. Improved by 0.255128.
Iteration 13. Log joint probability =    108.655. Improved by 0.92564.
Iteration 14. Log joint probabili

SystemError: <class 'stanfit4anon_model_dfdaf2b8ece8a02eb11f050ec701c0ec_7314420562355698783.PyStanHolder'> returned a result with an error set

In [ ]:
future = my_model.make_future_dataframe(periods=7)
future

In [ ]:
predict= my_model.predict(future)
predict[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(8)

In [ ]:
my_model.plot(predict, uncertainty=True)

In [ ]:
my_model.plot_components(predict)